In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('train.csv')
df

,metaphorID,label_boolean,text
0,0,True,"Hey , Karen !!!! I was told that on the day of..."
1,2,False,"Hi Ladies ... my last chemo was Feb 17/09 , ra..."
2,2,False,I have just come form my consult with a lovely...
3,4,False,I also still question taking Tamox for stage 1...
4,2,False,Just checking in to say hello ladies . I had a...
...,...,...,...
1865,4,True,Hi there . I found my lump 3 weeks ago and it ...
1866,4,True,Robyn-Sorry you find yourself on this web site...
1867,0,True,I 'm happy Jule that you posted this question ...
1868,5,True,Hiya April RADs-I should probably have been he...


In [3]:
metaphor_ID_to_word ={
    0: 'road',
    1: 'candle',
    2: 'light',
    3: 'spice',
    4: 'ride',
    5: 'train',
    6: 'boat'
}

df['metaphor_word'] = df['metaphorID'].map(metaphor_ID_to_word)
df

,metaphorID,label_boolean,text,metaphor_word
0,0,True,"Hey , Karen !!!! I was told that on the day of...",road
1,2,False,"Hi Ladies ... my last chemo was Feb 17/09 , ra...",light
2,2,False,I have just come form my consult with a lovely...,light
3,4,False,I also still question taking Tamox for stage 1...,ride
4,2,False,Just checking in to say hello ladies . I had a...,light
...,...,...,...,...
1865,4,True,Hi there . I found my lump 3 weeks ago and it ...,ride
1866,4,True,Robyn-Sorry you find yourself on this web site...,ride
1867,0,True,I 'm happy Jule that you posted this question ...,road
1868,5,True,Hiya April RADs-I should probably have been he...,train


In [30]:
import nltk
def identify_metaphor_sentence(txt, metaphor):
    for sentence in nltk.sent_tokenize(txt):
        if ' ' + metaphor + ' ' in sentence:
            return sentence

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset

# Load the dataset
df = pd.read_csv("train.csv")

# Drop rows with missing values
df = df.dropna(subset=['text', 'label_boolean'])

#replace metophorids with words
metaphor = {0:'road', 1:'candle', 2:'light', 3:'spice', 4:'ride', 5:'train', 6:'boat'}
df.replace({"metaphorID": metaphor},inplace=True)
# replace the text with the first sentence which contains the metaphor word
df['text'] = df.apply(lambda x: identify_metaphor_sentence(x['text'], x['metaphorID']),axis=1)

#Drop rows with missing values
df = df.dropna(subset=['text', 'label_boolean'])
df

,metaphorID,label_boolean,text
0,road,True,"You will do great , just a gliche in the road ."
1,light,False,so now my question ... are any of you into run...
2,light,False,I have just had a gin and tonic -LRB- strong o...
3,ride,False,Is n't the rads to get ride of any stray cance...
5,light,True,The light at the end of the tunnel keeps getti...
...,...,...,...
1864,road,True,"I mean , if I waited and if mets showed up 3 m..."
1865,ride,True,I found my lump 3 weeks ago and it 's been a s...
1866,ride,True,It is like a roller coaster ride at first .
1867,road,True,So I 'll definitely be talking to my onc down ...


In [32]:
from transformers import BertTokenizer, BertModel
import torch

def get_word_embeddings(sentence, metaphor_word):
    # Load pre-trained BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Tokenize the input sentence
    tokens = tokenizer(sentence, return_tensors='pt')

    # Get the index of the metaphor word in the tokenized sequence
    metaphor_token_index = tokens['input_ids'][0].tolist().index(tokenizer.convert_tokens_to_ids(metaphor_word))

    # Forward pass to get hidden states
    with torch.no_grad():
        outputs = model(**tokens)

    # Extract embeddings for the metaphor word
    metaphor_word_embeddings = outputs.last_hidden_state[:, metaphor_token_index, :].numpy()

    return metaphor_word_embeddings



In [33]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

def get_word_embeddings1(sentence, metaphor_word):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Try to convert the metaphor word to token ID
    try:
        metaphor_token_id = tokenizer.convert_tokens_to_ids(metaphor_word)
    except:
        print(f"Warning: '{metaphor_word}' is not in the tokenizer's vocabulary.")
        return None

    tokens = tokenizer(sentence, return_tensors='pt')
    
    # Check if the metaphor token ID is in the tokenized sequence
    if metaphor_token_id not in tokens['input_ids'][0].tolist():
        print(f"Warning: '{metaphor_word}' not found in the tokenized sequence.in sentece {sentence}")
        return None

    metaphor_token_index = tokens['input_ids'][0].tolist().index(metaphor_token_id)

    with torch.no_grad():
        outputs = model(**tokens)

    metaphor_word_embeddings = outputs.last_hidden_state[:, metaphor_token_index, :].numpy()

    return metaphor_word_embeddings.squeeze()

In [34]:
# Function to apply on each row of the DataFrame
def embed_row(row):
    metaphor_word = row['metaphorID']
    sentence = row['text']
    embeddings = get_word_embeddings1(sentence, metaphor_word)
    return embeddings.squeeze()

In [35]:
# Apply the function to each row
df['Embeddings'] = df.apply(embed_row, axis=1)